In [4]:
!pip install roboflow
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.7 MB/s eta 0:00:00
  

In [5]:
from google.colab import drive
from ultralytics import YOLO
import os
import cv2
import numpy as np
from glob import glob

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Nieuwe outputmap Drive
data_path = "/content/drive/MyDrive/roboflow_dataset_detectie"
os.makedirs(data_path, exist_ok=True)

os.chdir(data_path)


In [ ]:
# Dataset roboflow
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="pDX5tjADYL9oViENUvwf")
project = rf.workspace("tata-steel-zabde").project("amr2-yviht")
version = project.version(9)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to AMR2-9 in yolov11:: 100%|██████████| 2896/2896 [00:37<00:00, 77.52it/s]


In [7]:
data_yaml_path = "/content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/data.yaml"

In [11]:
model_path = "/content/detectie_model.tflite"
model = YOLO(model_path, task='detect')

In [12]:
print(model.names)
print("Aantal klassen:", len(model.names))

Loading /content/detectie_model.tflite for TensorFlow Lite inference...
{0: 'display'}
Aantal klassen: 1


# Validatie

In [ ]:
# AP op validatie
metrics_val = model.val(data=data_yaml_path, split='val', imgsz=160, save=False, save_json=False, save_txt=True, plots=False)
print("Validatie-resultaten:")
mp, mr, map50, map = metrics_val.box.mean_results()
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map:.4f}")



Ultralytics 8.3.143 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Loading /content/detectie_model.tflite for TensorFlow Lite inference...
Setting batch=1 input of shape (1, 3, 160, 160)
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 8.5±3.0 MB/s, size: 33.8 KB)


val: Scanning /content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/valid/labels... 287 images, 5 backgrounds, 0 corrupt: 100%|██████████| 287/287 [00:06<00:00, 47.44it/s]


val: New cache created: /content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/valid/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 287/287 [00:13<00:00, 21.65it/s]

                   all        287        286      0.985      0.943      0.985      0.661
Speed: 0.2ms preprocess, 15.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Validatie-resultaten:
mAP@0.5: 0.9847
mAP@0.5:0.95: 0.6614


In [ ]:
float = __builtins__.float
int = __builtins__.int

def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    return interArea / (boxAArea + boxBArea - interArea + 1e-6)

# Pad-instellingen
img_dir = '/content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/valid/images'
label_dir = '/content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/valid/labels'
output_dir = '/content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/mismatches_val'
os.makedirs(output_dir, exist_ok=True)

image_paths = glob(os.path.join(img_dir, '*.jpg'))

true_positives = 0
total_gt_boxes = 0
iou_threshold = 0.5

for img_path in image_paths:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(label_dir, filename + '.txt')

    # Lees afbeelding
    img = cv2.imread(img_path)
    height, width = img.shape[:2]

    # Voorspelling
    results = model(img)
    boxes = results[0].boxes

    if boxes is None or len(boxes) == 0:
        continue

    # Kies box met hoogste confidence
    confs = boxes.conf.cpu().numpy()
    idx_best = np.argmax(confs)
    pred_box = boxes.xyxy[idx_best].cpu().numpy()

    # Laad ground truth
    if not os.path.exists(label_path):
        continue

    with open(label_path, 'r') as f:
        lines = f.readlines()

    gt_boxes = []
    for line in lines:
        parts = [float(x) for x in line.strip().split()]
        if len(parts) != 5:
            continue
        cls, x_c, y_c, w, h = parts
        x1 = (x_c - w / 2) * width
        y1 = (y_c - h / 2) * height
        x2 = (x_c + w / 2) * width
        y2 = (y_c + h / 2) * height
        gt_boxes.append([x1, y1, x2, y2])

    total_gt_boxes += len(gt_boxes)

    # Vergelijk voorspelling met GT-boxen
    match_found = any(iou(pred_box, gt_box) >= iou_threshold for gt_box in gt_boxes)
    if match_found:
        true_positives += 1
    else:
        # Teken GT-boxen (groen)
        for gt in gt_boxes:
            gx1, gy1, gx2, gy2 = [int(x) for x in gt]
            cv2.rectangle(img, (gx1, gy1), (gx2, gy2), (0, 255, 0), 2)

        # Teken voorspelde box (rood)
        px1, py1, px2, py2 = [int(x) for x in pred_box]
        cv2.rectangle(img, (px1, py1), (px2, py2), (0, 0, 255), 2)

        # Sla mismatch-afbeelding op
        save_path = os.path.join(output_dir, f'{filename}_mismatch.jpg')
        cv2.imwrite(save_path, img)

# Bereken recall
recall = true_positives / total_gt_boxes if total_gt_boxes > 0 else 0
print(f"Handmatig berekende recall: {recall:.4f}")
print(f"Mismatches opgeslagen in map: {output_dir}")



0: 160x160 (no detections), 21.1ms
Speed: 1.7ms preprocess, 21.1ms inference, 1.2ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.5ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 30.9ms
Speed: 1.3ms preprocess, 30.9ms inference, 5.3ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 (no detections), 14.0ms
Speed: 1.2ms preprocess, 14.0ms inference, 0.8ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 10.3ms
Speed: 2.2ms preprocess, 10.3ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 11.1ms
Speed: 0.9ms preprocess, 11.1ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 10.4ms
Speed: 2.7ms preprocess, 10.4ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 10.3ms
Speed: 0.9ms preprocess, 10.3ms inference, 1.1ms postprocess per imag

# Recall: 96.45

# Test

In [ ]:
# AP op validatie
metrics_val = model.val(data=data_yaml_path, split='test', imgsz=160, save=False, save_json=False, save_txt=True, plots=False)
print("test-resultaten:")
mp, mr, map50, map = metrics_val.box.mean_results()
print(f"mAP@0.5: {map50:.4f}")
print(f"mAP@0.5:0.95: {map:.4f}")



Ultralytics 8.3.143 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Loading /content/detectie_model.tflite for TensorFlow Lite inference...
Setting batch=1 input of shape (1, 3, 160, 160)
val: Fast image access ✅ (ping: 1.0±0.9 ms, read: 14.4±6.6 MB/s, size: 55.9 KB)


val: Scanning /content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/test/labels... 290 images, 4 backgrounds, 0 corrupt: 100%|██████████| 290/290 [00:04<00:00, 70.48it/s]

val: New cache created: /content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/test/labels.cache


WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1, len(boxes) = 288. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 290/290 [00:18<00:00, 15.44it/s]

                   all        290        288      0.964      0.934      0.979      0.654
Speed: 0.3ms preprocess, 26.2ms inference, 0.0ms loss, 2.0ms postprocess per image
test-resultaten:
mAP@0.5: 0.9788
mAP@0.5:0.95: 0.6535


In [ ]:
float = __builtins__.float
int = __builtins__.int

def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    return interArea / (boxAArea + boxBArea - interArea + 1e-6)

# Pad-instellingen
img_dir = '/content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/test/images'
label_dir = '/content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/test/labels'
output_dir = '/content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/mismatches_test'
os.makedirs(output_dir, exist_ok=True)

image_paths = glob(os.path.join(img_dir, '*.jpg'))

true_positives = 0
total_gt_boxes = 0
iou_threshold = 0.5

for img_path in image_paths:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(label_dir, filename + '.txt')

    # Lees afbeelding
    img = cv2.imread(img_path)
    height, width = img.shape[:2]

    # Voorspelling
    results = model(img)
    boxes = results[0].boxes

    if boxes is None or len(boxes) == 0:
        continue

    # Kies box met hoogste confidence
    confs = boxes.conf.cpu().numpy()
    idx_best = np.argmax(confs)
    pred_box = boxes.xyxy[idx_best].cpu().numpy()

    # Laad ground truth
    if not os.path.exists(label_path):
        continue

    with open(label_path, 'r') as f:
        lines = f.readlines()

    gt_boxes = []
    for line in lines:
        parts = [float(x) for x in line.strip().split()]
        if len(parts) != 5:
            continue
        cls, x_c, y_c, w, h = parts
        x1 = (x_c - w / 2) * width
        y1 = (y_c - h / 2) * height
        x2 = (x_c + w / 2) * width
        y2 = (y_c + h / 2) * height
        gt_boxes.append([x1, y1, x2, y2])

    total_gt_boxes += len(gt_boxes)

    # Vergelijk voorspelling met GT-boxen
    match_found = any(iou(pred_box, gt_box) >= iou_threshold for gt_box in gt_boxes)
    if match_found:
        true_positives += 1
    else:
        # Teken GT-boxen (groen)
        for gt in gt_boxes:
            gx1, gy1, gx2, gy2 = [int(x) for x in gt]
            cv2.rectangle(img, (gx1, gy1), (gx2, gy2), (0, 255, 0), 2)

        # Teken voorspelde box (rood)
        px1, py1, px2, py2 = [int(x) for x in pred_box]
        cv2.rectangle(img, (px1, py1), (px2, py2), (0, 0, 255), 2)

        # Sla mismatch-afbeelding op
        save_path = os.path.join(output_dir, f'{filename}_mismatch.jpg')
        cv2.imwrite(save_path, img)

# Bereken recall
recall = true_positives / total_gt_boxes if total_gt_boxes > 0 else 0
print(f"Handmatig berekende recall: {recall:.4f}")
print(f"Mismatches opgeslagen in map: {output_dir}")



0: 160x160 1 display, 27.7ms
Speed: 1.1ms preprocess, 27.7ms inference, 1.5ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 19.1ms
Speed: 0.8ms preprocess, 19.1ms inference, 1.4ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 19.5ms
Speed: 0.9ms preprocess, 19.5ms inference, 2.2ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 20.4ms
Speed: 0.8ms preprocess, 20.4ms inference, 1.6ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 21.7ms
Speed: 0.8ms preprocess, 21.7ms inference, 2.1ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 24.0ms
Speed: 0.9ms preprocess, 24.0ms inference, 2.2ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 18.6ms
Speed: 0.8ms preprocess, 18.6ms inference, 1.6ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 18.2ms
Speed: 0.8ms preprocess, 18.2ms inference, 1.7ms postprocess per image at shape (

# Recall: 97.18

In [23]:
import random

# Map met testafbeeldingen
img_dir = '/content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/test/images'
output_random_dir = '/content/drive/MyDrive/roboflow_dataset_detectie/AMR2-9/random_detectie'
os.makedirs(output_random_dir, exist_ok=True)

# Selecteer 10 willekeurige afbeeldingen
all_images = glob(os.path.join(img_dir, '*.jpg'))
random_images = random.sample(all_images, min(10, len(all_images)))

for img_path in random_images:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    img = cv2.imread(img_path)

    results = model(img)
    boxes = results[0].boxes

    if boxes is not None and len(boxes) > 0:
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].item()
            cls_id = int(box.cls[0].item())
            label = f'{cls_id} {conf:.2f}'
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 0), 2)
            cv2.putText(img, label, (int(x1), int(y1) - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

    save_path = os.path.join(output_random_dir, f'{filename}_detectie.jpg')
    cv2.imwrite(save_path, img)

print(f"Willekeurige detecties opgeslagen in map: {output_random_dir}")



0: 160x160 1 display, 15.6ms
Speed: 0.9ms preprocess, 15.6ms inference, 1.4ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 10.8ms
Speed: 0.9ms preprocess, 10.8ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 12.7ms
Speed: 0.8ms preprocess, 12.7ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 10.2ms
Speed: 1.3ms preprocess, 10.2ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 11.4ms
Speed: 1.1ms preprocess, 11.4ms inference, 1.4ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 10.6ms
Speed: 1.1ms preprocess, 10.6ms inference, 1.1ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 12.4ms
Speed: 0.8ms preprocess, 12.4ms inference, 1.6ms postprocess per image at shape (1, 3, 160, 160)

0: 160x160 1 display, 11.2ms
Speed: 1.2ms preprocess, 11.2ms inference, 1.3ms postprocess per image at shape (